In [1]:
import torch
import torchvision
from torch import nn


from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform
from efficientnet_pytorch import EfficientNet


class SwaV(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(512, 512, 128)
        self.prototypes = SwaVPrototypes(128, n_prototypes=512)

    def forward(self, x):
        x = self.backbone(x).flatten(start_dim=1)
        x = self.projection_head(x)
        x = nn.functional.normalize(x, dim=1, p=2)
        p = self.prototypes(x)
        return p


resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = SwaV(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


backbone = EfficientNet.from_pretrained('efficientnet-b0')
# Ignore the classification head as we only want the features.
backbone._fc = torch.nn.Identity()

Loaded pretrained weights for efficientnet-b0


In [6]:
!pipenv install huggingface transformers==4.14.1 tokenizers==0.10.3 

Installing -c...
Resolving -c...
[    ] Installing...
[    ] Installing...



Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Traceback (most recent call last):
  File "C:\Users\Maods\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\pipenv\patched\pip\_vendor\packaging\requirements.py", line 102, in __init__
    req = REQUIREMENT.parseString(requirement_string)
  File "C:\Users\Maods\.pyenv\pyenv-win\versions\3.10.4\lib\site-packages\pipenv\patched\pip\_vendor\pyparsing\core.py", line 1141, in parse_string
    raise exc.with_traceback(None)
pipenv.patched.pip._vendor.pyparsing.exceptions.ParseException: Expected W:(0-9A-Za-z), found '-'  (at char 0), (line:1, col:1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "

In [18]:
# Create a PyTorch module for the SimCLR model.
class SimCLR(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = heads.SimCLRProjectionHead(
            input_dim=512,  # Resnet18 features have 512 dimensions.
            hidden_dim=512,
            output_dim=128,
        )

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(features)
        return z


# Use a resnet backbone.
backbone = EfficientNet.from_pretrained('efficientnet-b0')
# Ignore the classification head as we only want the features.
backbone._fc = torch.nn.Identity()

# Build the SimCLR model.
model = SimCLR(backbone)




Loaded pretrained weights for efficientnet-b0


In [23]:
# Create a PyTorch module for the SimCLR model.
class SimCLR(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.projection_head = heads.SimCLRProjectionHead(
            input_dim=1280,  # Resnet18 features have 512 dimensions.
            hidden_dim=1280,
            output_dim=128,
        )

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(features)
        return z


# Use a resnet backbone.
backbone = EfficientNet.from_pretrained('efficientnet-b0')
# Ignore the classification head as we only want the features.
backbone._fc = torch.nn.Identity()

# Build the SimCLR model.
model = SimCLR(backbone)


# Prepare transform that creates multiple random views for every image.
transform = transforms.SimCLRTransform(input_size=32, cj_prob=0.5)


# Create a dataset from your image folder.
dataset = LightlyDataset(input_dir="../data/02_data_split/train_data/", transform=transform)

# Build a PyTorch dataloader.
dataloader = torch.utils.data.DataLoader(
    dataset,  # Pass the dataset to the dataloader.
    batch_size=128,  # A large batch size helps with the learning.
    shuffle=True,  # Shuffling is important!
)

# Lightly exposes building blocks such as loss functions.
criterion = loss.NTXentLoss(temperature=0.5)

# Get a PyTorch optimizer.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-6)

# Train the model.
for epoch in range(10):
    for (view0, view1), targets, filenames in dataloader:
        z0 = model(view0)
        z1 = model(view1)
        loss = criterion(z0, z1)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"loss: {loss.item():.5f}")

Loaded pretrained weights for efficientnet-b0
loss: 5.50475
loss: 5.51909
loss: 5.52904
loss: 5.51790
loss: 5.53173
loss: 5.49575
loss: 5.51858
loss: 5.53091
loss: 5.48558
loss: 5.46340
loss: 5.45240
loss: 5.47274
loss: 5.50354
loss: 5.50478
loss: 5.49191
loss: 5.47412
loss: 5.44785
loss: 5.42565
loss: 5.51401
loss: 5.45893
loss: 5.44463
loss: 5.43077
loss: 5.41673
loss: 5.47465
loss: 5.43301
loss: 5.41797
loss: 5.43376
loss: 5.45916
loss: 5.38749
loss: 5.41056
loss: 5.35334
loss: 5.35617
loss: 5.42437
loss: 5.40366
loss: 5.38953
loss: 5.35424
loss: 5.41400
loss: 5.35691
loss: 5.41279
loss: 5.42426
loss: 5.45534
loss: 5.37404
loss: 5.36493
loss: 5.34767
loss: 5.40132
loss: 5.31602
loss: 5.37155
loss: 5.38717
loss: 5.34566
loss: 5.36895
loss: 5.35160
loss: 5.42077
loss: 5.33988
loss: 5.32502
loss: 5.31049
loss: 5.33960
loss: 5.36135
loss: 5.29119
loss: 5.35251
loss: 5.29370
loss: 5.27821
loss: 5.31126
loss: 5.33855
loss: 5.30880
loss: 5.26585
loss: 5.25315
loss: 5.31130
loss: 5.26712
lo